## Generate xESMF-compatible Horizontal Grid for OM4

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import xesmf
import momgrid
import matplotlib.pyplot as plt

Load the `ocean_hgrid.nc` file but use a static file from the model for the depth/topography field. (OM4 used runtime edits that are not in the gridspec!)

In [3]:
hgrid = "ocean_hgrid.nc"
static = "ocean.static.nc"

In [4]:
grid = momgrid.MOMgrid(hgrid, topog=static, depth_var="deptho", symmetric=True)

In [5]:
dsout = grid.to_xesmf()

dsout["xh"] = dsout["xh"].astype("float32")
dsout["yh"] = dsout["yh"].astype("float32")

dsout["xq"] = dsout["xq"].astype("float32")
dsout["yq"] = dsout["yq"].astype("float32")


# set attributes
dsout["xh"].attrs = {
    "units": "1",
    "long_name": "h point nominal x-index",
}

dsout["yh"].attrs = {
    "units": "1",
    "long_name": "h point nominal y-index",
}

dsout["xq"].attrs = {
    "units": "1",
    "long_name": "q point nominal x-index",
}

dsout["yq"].attrs = {
    "units": "1",
    "long_name": "q point nominal y-index",
}

dsout["lat"].attrs = {
    "units": "degrees_east",
    "standard_name": "latitude",
    "long_name": "Latitude of tracer (T) points",
    "axis": "Y",
    "cartesian_axis": "Y",
    "bounds": "lat_b",
}

dsout["lon"].attrs = {
    "units": "degrees_east",
    "standard_name": "longitude",
    "long_name": "Longitude of tracer (T) points",
    "axis": "X",
    "cartesian_axis": "X",
    "bounds": "lon_b",
}

dsout["mask"].attrs = {
    "standard_name": "sea_binary_mask",
	"long_name": "Sea Binary Mask 1 = sea, 0 = land",
	"units": "1",
}

dsout["lat_b"].attrs = {
	"long_name": "Latitude bounds",
    "comment": "Values define the corner points"
}

dsout["lon_b"].attrs = {
	"long_name": "Longitude bounds",
    "comment": "Values define the corner points"
}

dsout = dsout.set_coords("lat")
dsout = dsout.set_coords("lon")

dsout["mask"] = dsout["mask"].astype("i1")

dsout.attrs = {
    "tile": "OM4 Horizontal Tracer Grid",
}

In [6]:
dsout

<xarray.Dataset>
Dimensions:  (xh: 1440, yh: 1080, xq: 1441, yq: 1081)
Coordinates:
  * xh       (xh) float32 0.5 1.5 2.5 3.5 ... 1.438e+03 1.438e+03 1.44e+03
  * yh       (yh) float32 0.5 1.5 2.5 3.5 ... 1.078e+03 1.078e+03 1.08e+03
    lat      (yh, xh) float32 -79.81 -79.81 -79.8 -79.8 ... 64.33 64.22 64.11
    lon      (yh, xh) float32 -299.7 -299.5 -299.2 -299.0 ... 60.0 60.0 60.0
  * xq       (xq) float32 1.0 2.0 3.0 4.0 ... 1.439e+03 1.44e+03 1.441e+03
  * yq       (yq) float32 1.0 2.0 3.0 4.0 ... 1.079e+03 1.08e+03 1.081e+03
Data variables:
    lat_b    (yq, xq) float32 -79.84 -79.84 -79.84 -79.83 ... 64.28 64.17 64.06
    lon_b    (yq, xq) float32 -299.8 -299.6 -299.3 -299.1 ... 60.0 60.0 60.0
    mask     (yh, xh) int8 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
Attributes:
    tile:     OM4 Horizontal Tracer Grid

In [7]:
dsout.to_netcdf("OM4_horiz_t_grid.nc")

In [8]:
!ls -lh OM4_horiz_t_grid.nc

-rw-r--r-- 1 John.Krasting o 26M Dec 14 12:55 OM4_horiz_t_grid.nc
